In [1]:
import numpy as np
import pandas as pd

<h1 align = 'center'>Ratio of each type/bucket in each CD</h1>

In [8]:
ratio08 = pd.read_csv('CB_bucket_prop_year_2008_nocount.csv')
ratio09 = pd.read_csv('CB_bucket_prop_year_2009_nocount.csv')
ratio10 = pd.read_csv('CB_bucket_prop_year_2010_nocount.csv')

In [9]:
ratio08.head(3)

,Unnamed: 0,CBCD,dual_elig,white,male,agecat,grand_total,count_over_CB_total,CB_total_over_grand_total,year
0,1,101,0,1,0,1,264378,0.211318,0.006283,2008
1,2,101,1,0,0,1,264378,0.198675,0.006283,2008
2,3,101,0,1,1,1,264378,0.102950,0.006283,2008


In [10]:
ratio08['type'] = ratio08.dual_elig * 8 + ratio08.white * 4 + ratio08.male * 2 + ratio08.agecat
ratio09['type'] = ratio09.dual_elig * 8 + ratio09.white * 4 + ratio09.male * 2 + ratio09.agecat
ratio10['type'] = ratio10.dual_elig * 8 + ratio10.white * 4 + ratio10.male * 2 + ratio10.agecat

In [12]:
ratio08['ratio_over_whole_NYC'] = ratio08.count_over_CB_total * ratio08.CB_total_over_grand_total
ratio09['ratio_over_whole_NYC'] = ratio09.count_over_CB_total * ratio09.CB_total_over_grand_total
ratio10['ratio_over_whole_NYC'] = ratio10.count_over_CB_total * ratio10.CB_total_over_grand_total

In [14]:
ratio_type_cd = pd.DataFrame(columns = ['CD', 'type', 'ratio_over_CD', 'ratio_over_whole_NYC'])

In [84]:
ratio_concat = pd.concat([ratio08, ratio09, ratio10], axis = 0).reset_index(drop=True)
ratio_concat.head(5)

,Unnamed: 0,CBCD,dual_elig,white,male,agecat,grand_total,count_over_CB_total,CB_total_over_grand_total,year,type,ratio_over_whole_NYC
0,1,101,0,1,0,1,264378,0.211318,0.006283,2008,5,0.001328
1,2,101,1,0,0,1,264378,0.198675,0.006283,2008,9,0.001248
2,3,101,0,1,1,1,264378,0.102950,0.006283,2008,7,0.000647
3,4,101,1,0,1,1,264378,0.083685,0.006283,2008,11,0.000526
4,5,101,0,0,0,1,264378,0.071644,0.006283,2008,1,0.000450


In [90]:
def f(x):
    d = {}
    d['ratio_over_CD'] = x['count_over_CB_total'].mean()
    d['CD_ratio_over_whole_NYC'] = x['CB_total_over_grand_total'].mean()
    return pd.Series(d, index=['ratio_over_CD', 'CD_ratio_over_whole_NYC'])

ratio = ratio_concat.groupby(['CBCD','type']).apply(f).reset_index().rename(columns = {'CBCD':'CD'})
ratio.head()

,CD,type,ratio_over_CD,CD_ratio_over_whole_NYC
0,101,0,0.025581,0.006294
1,101,1,0.072090,0.006294
2,101,2,0.018781,0.006294
3,101,3,0.036297,0.006294
4,101,4,0.045929,0.006294


In [91]:
ratio['ratio_over_whole_NYC'] = ratio.ratio_over_CD * ratio.CD_ratio_over_whole_NYC
ratio.head(5)

,CD,type,ratio_over_CD,CD_ratio_over_whole_NYC,ratio_over_whole_NYC
0,101,0,0.025581,0.006294,0.000161
1,101,1,0.072090,0.006294,0.000454
2,101,2,0.018781,0.006294,0.000118
3,101,3,0.036297,0.006294,0.000228
4,101,4,0.045929,0.006294,0.000289


In [93]:
ratio_CD_over_whole_NYC = ratio_concat.groupby(['CBCD','year'])['CB_total_over_grand_total'].mean().reset_index().groupby('CBCD')['CB_total_over_grand_total'].mean().reset_index().rename(columns = {'CBCD':'CD'})
ratio_CD_over_whole_NYC.head()

,CD,CB_total_over_grand_total
0,101,0.006294
1,102,0.011995
2,103,0.024740
3,104,0.011593
4,105,0.006842


In [98]:
def f(x):
    d = {}
    d['type_ratio_over_whole_NYC'] = x['ratio_over_whole_NYC'].sum()
    d['type_ratio_max_among_CD'] = x['ratio_over_whole_NYC'].max()
    d['type_ratio_min_among_CD'] = x['ratio_over_whole_NYC'].min()
    return pd.Series(d, index=['type_ratio_over_whole_NYC','type_ratio_max_among_CD','type_ratio_min_among_CD'])

ratio_type_over_whole_NYC = ratio.groupby(['type']).apply(f).reset_index()
ratio_type_over_whole_NYC

,type,type_ratio_over_whole_NYC,type_ratio_max_among_CD,type_ratio_min_among_CD
0,0,0.037997,0.002413,0.000122
1,1,0.047187,0.003704,0.000138
2,2,0.024489,0.001760,0.000087
3,3,0.024885,0.001607,0.000070
4,4,0.069439,0.005825,0.000004
5,5,0.194109,0.014438,0.000005
6,6,0.058197,0.004586,0.000004
7,7,0.118639,0.010142,0.000004
8,8,0.056822,0.003355,0.000139
9,9,0.121113,0.007400,0.000264


In [119]:
ratio_CD_within_each_type = ratio.groupby(['type', 'CD']).agg({'ratio_over_whole_NYC': 'sum'}).groupby(level=0).apply(lambda x: x / float(x.sum())).rename(columns = {'ratio_over_whole_NYC':'ratio_CD_within_type'}).reset_index()
ratio_CD_within_each_type.head()

,type,CD,ratio_CD_within_type
0,0,101,0.004238
1,0,102,0.003787
2,0,103,0.017274
3,0,104,0.009110
4,0,105,0.003552


In [120]:
ratio.to_csv('ratio.csv', index = False)
ratio_CD_over_whole_NYC.to_csv('ratio_CD_over_whole_NYC.csv', index = False)
ratio_type_over_whole_NYC.to_csv('ratio_type_over_whole_NYC.csv', index = False)
ratio_CD_within_each_type.to_csv('ratio_CD_within_each_type.csv', index = False)

<h1 align = "center"> Generate initial weights table </h1>

In [28]:
weights = pd.DataFrame(columns = ['agent-type', 'choice-type', "attribute", "social_norm","efficacy"])

In [29]:
for i in range(16):
    for j in ['ER', 'Office']:
        weights = weights.append({'agent-type':i,'choice-type':j,"attribute":1,"social_norm":1,"efficacy":1}, ignore_index = True)

In [30]:
weights.to_csv('weights.csv',index = False)

<h1 align = "center"> Generate Gamma Distribution Matrix for LOS </h1>

In [2]:
from scipy.stats import gamma
import matplotlib.pyplot as plt

In [3]:
gamma_los = pd.read_csv("hospitalization_los_distribution_params_all_bucket_whole_city.csv")
gamma_los = gamma_los.drop(gamma_los.columns[0],axis = 1)

In [4]:
gamma_los[['bucket','n_mixture']]

,bucket,n_mixture
0,0,4
1,1,8
2,10,2
3,11,6
4,100,7
5,101,8
6,110,2
7,111,3
8,1000,6
9,1001,8


In [5]:
def f(row):
    agent_type = int(str(int(row['bucket'])),2)
    age = (agent_type & 0b1000) >> 3
    gender = (agent_type & 0b100) >> 2
    race = (agent_type & 0b10) >> 1
    dual_eligible = (agent_type & 0b1) >> 0
    agent_type = int(dual_eligible * 8 + race * 4 + gender * 2 + age)
    return pd.Series({'type':agent_type, 'dual-eligible':dual_eligible, 'race': race, 'gender': gender, 'age': age}, dtype = np.int32)
    
gamma_los[['type','dual-eligible','race','gender','age']] = gamma_los.apply(f, axis = 1)
gamma_los.head()

,bucket,n_mixture,lambda_1,alpha_1,beta_1,lambda_2,alpha_2,beta_2,lambda_3,alpha_3,...,alpha_8,beta_8,lambda_9,alpha_9,beta_9,type,dual-eligible,race,gender,age
0,0,4,0.399180,6.512051,0.216114,0.369348,11.507293,0.391921,0.118926,27.288766,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
1,1,8,0.222673,16.117042,0.066902,0.195480,24.221707,0.097224,0.169722,36.964258,...,1.402935,63.877074,NaN,NaN,NaN,8,1,0,0,0
2,10,2,0.268304,8.064436,0.160551,0.731696,0.839592,10.355997,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,0,1,0,0
3,11,6,0.240983,49.798034,0.020511,0.124704,24.807690,0.093375,0.077833,88.121480,...,NaN,NaN,NaN,NaN,NaN,12,1,1,0,0
4,100,7,0.286704,53.012782,0.019241,0.217655,24.088971,0.100857,0.077138,91.585133,...,NaN,NaN,NaN,NaN,NaN,2,0,0,1,0


In [8]:
class los:
    def __init__(self, raw_df):
        self.raw_df = raw_df
        self.los = pd.DataFrame(columns = ['type','day','ratio'])
        
    def get_discrete_gamma(self, shape, scale, epsilon):
        last_cdf = 0
        new_cdf = 0
        i = 0
        result = {}
        while new_cdf < 1 - epsilon:
            new_cdf = gamma.cdf(i + 0.5, a = shape, scale = scale)
            result[i] = new_cdf - last_cdf
            last_cdf = new_cdf
            i += 1
        return result
    
    def process_row(self, row):
        n_mixture = row['n_mixture']
        for i in np.arange(1, n_mixture + 1, dtype = np.int32):
            this_gamma_ratio = row['lambda_' + repr(i)]
            this_gamma_result = self.get_discrete_gamma(shape = row['alpha_' + repr(i)], scale = row['beta_' + repr(i)], epsilon = 0.001)
            for key, value in this_gamma_result.items():
                self.los = self.los.append({'type':row['type'], 'day':key, 'ratio':value * this_gamma_ratio}, ignore_index = True)
        
    def generate_los(self, threshold = 0.003):
        self.los = pd.DataFrame(columns = ['type','day','ratio'])
        self.raw_df.apply(lambda row: self.process_row(row), axis = 1)
        self.los = self.los.groupby(['type', 'day']).sum().query('ratio > '+ repr(threshold)).groupby(level=0).apply(lambda x: x / float(x.sum())).sort_values(by = ['type', 'ratio'], ascending = [True, False]).reset_index()
        self.los['order'] = self.los.groupby('type').cumcount()
        self.los['type'] = self.los['type'].astype(int)
        self.los['day'] = self.los['day'].astype(int)
        
    def parse_row(self, row):
            return [[row['day'], row['ratio']], ['day' + repr(int(row['order'])), 'ratio' + repr(int(row['order']))]]
    
    def generate_los_concentrated(self):
        self.los_conc = pd.DataFrame()
        for the_type in self.los['type'].unique():
            parsed_rows = self.los[self.los['type'] == the_type].apply(self.parse_row, axis = 1)
            this_row = pd.DataFrame({'type':[the_type], 'ndays': len(parsed_rows)})
            for row_list in parsed_rows:
                this_row = pd.concat([this_row,pd.DataFrame(data = [row_list[0]], columns = row_list[1])], axis = 1, sort = False)
            self.los_conc = self.los_conc.append(this_row, sort = False)
        self.los_conc['type'] = self.los_conc['type'].astype(int)

In [9]:
x = los(gamma_los)
x.generate_los(0.003)
x.generate_los_concentrated()

In [10]:
x.los.head(20)

,type,day,ratio,order
0,0,1,0.258917,0
1,0,2,0.166361,1
2,0,4,0.123591,2
3,0,5,0.102326,3
4,0,3,0.095431,4
5,0,6,0.058315,5
6,0,7,0.030469,6
7,0,10,0.029023,7
8,0,9,0.026692,8
9,0,11,0.026211,9


In [11]:
x.los_conc

,type,ndays,day0,ratio0,day1,ratio1,day2,ratio2,day3,ratio3,...,day29,ratio29,day30,ratio30,day31,ratio31,day32,ratio32,day33,ratio33
0,0,17,1.0,0.258917,2.0,0.166361,4.0,0.123591,5.0,0.102326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,22,1.0,0.224659,4.0,0.124631,2.0,0.122569,3.0,0.113942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,28,1.0,0.305487,2.0,0.132689,3.0,0.112629,4.0,0.085395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3,29,1.0,0.227690,2.0,0.150574,3.0,0.121654,4.0,0.088823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,30,1.0,0.278573,2.0,0.147229,0.0,0.065277,3.0,0.063083,...,29.0,0.003348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5,26,2.0,0.163954,1.0,0.141119,3.0,0.139853,4.0,0.107516,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6,29,1.0,0.322455,2.0,0.147067,0.0,0.067720,3.0,0.058553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,7,19,1.0,0.273507,3.0,0.133333,2.0,0.111075,4.0,0.093302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8,27,1.0,0.217397,2.0,0.142578,3.0,0.127509,5.0,0.098118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,9,28,1.0,0.176839,3.0,0.140079,2.0,0.120531,4.0,0.109557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
x.los_conc.to_csv('los.csv', index = False)

<h1 align = "center"> Probability of Hospitalization after decision </h1>

In [17]:
prob = pd.read_csv("hosp_prop_after_decision_all_bucket_whole_city.csv")
prob = prob.drop(prob.columns[0],axis = 1)
prob.head()

,ER_hosp_prob,ER_hosp_ACSC_prob,EM_hosp_prob,EM_hosp_ACSC_prob,bucket
0,0.004488,0.411765,0.001437,0.180587,0
1,0.005781,0.181818,0.001955,0.262000,1
2,0.007937,0.229730,0.001467,0.220662,10
3,0.010690,0.187500,0.001404,0.229577,11
4,0.006157,0.192308,0.002036,0.170732,100


In [18]:
def f(row):
    agent_type = int(str(int(row['bucket'])),2)
    age = (agent_type & 0b1000) >> 3
    gender = (agent_type & 0b100) >> 2
    race = (agent_type & 0b10) >> 1
    dual_eligible = (agent_type & 0b1) >> 0
    agent_type = int(dual_eligible * 8 + race * 4 + gender * 2 + age)
    return pd.Series({'type':agent_type, 'dual-eligible':dual_eligible, 'race': race, 'gender': gender, 'age': age}, dtype = np.int32)
    
prob[['type','dual-eligible','race','gender','age']] = prob.apply(f, axis = 1)
prob.head()

,ER_hosp_prob,ER_hosp_ACSC_prob,EM_hosp_prob,EM_hosp_ACSC_prob,bucket,type,dual-eligible,race,gender,age
0,0.004488,0.411765,0.001437,0.180587,0,0,0,0,0,0
1,0.005781,0.181818,0.001955,0.262000,1,8,1,0,0,0
2,0.007937,0.229730,0.001467,0.220662,10,4,0,1,0,0
3,0.010690,0.187500,0.001404,0.229577,11,12,1,1,0,0
4,0.006157,0.192308,0.002036,0.170732,100,2,0,0,1,0


In [19]:
prob_to_save = prob[['type','ER_hosp_prob','EM_hosp_prob']]
prob_to_save

In [21]:
prob_to_save.to_csv('hosp_prob_after_decision.csv', index = False)